## 3(a) Linear Programming Model

The variables for the problem are defined as follows, <br>
$x_0$ - Amount of 2 year loan at $1\%$ interest per quarter borrowed in $Q_1$ <br>
$x_{3i-2}$ - Amount of 6 month loan at $1.8\%$ interest borrowed in $Q_i$ <br>
$x_{3i-1}$ - Amount of 3 month loan at $2.5\%$ interest borrowed in $Q_i$ <br>
$x_{3i}$ - Amount of cash surplus available after $Q_i$ <br>

The modelling idea is from [1]. There are 25 variables and 8 equality constraints for the given problem. We assume that the loans are repaid by equated monthly instalments (EMI). Using the formula $P*R*\frac{(1+R)^N}{(1+R)^N-1}$, we calculate the EMI for each type of loan and prorate for each quarter and add it as part of demand. The standard form for this LP is given below, <br>

<img src="diagram.png"> 

$$\min 0.510552x_{22}-1.005x_{24}$$

$$0.86974x_0 + 0.489448x_1 - 0.016709x_2 -x_3 = 100$$
$$-0.13026x_0 - 0.510552x_1 + 1.005x_3 + 0.489448x_4 - 0.016709x_5 -x_6 = 500$$
$$-0.13026x_0 - 0.510552x_4 + 1.005x_6 + 0.489448x_7 - 0.016709x_8 -x_9 = 100$$
$$-0.13026x_0 - 0.510552x_7 + 1.005x_9 + 0.489448x_{10} - 0.016709x_{11} -x_{12} = -600$$
$$-0.13026x_0 - 0.510552x_{10} + 1.005x_{12} + 0.489448x_{13} - 0.016709x_{14} -x_{15} = -500$$
$$-0.13026x_0 - 0.510552x_{13} + 1.005x_{15} + 0.489448x_{16} - 0.016709x_{17} -x_{18} = 200$$
$$-0.13026x_0 - 0.510552x_{16} + 1.005x_{18} + 0.489448x_{19} - 0.016709x_{20} -x_{21} = 600$$
$$-0.13026x_0 - 0.510552x_{19} + 1.005x_{21} + 0.489448x_{22} - 0.016709x_{23} -x_{24} = -900$$

$$x_i \geq 0$$

## 3(b) Revised Simplex Algorithm

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import time

In [3]:
def revised_simplex(c, A, b, max_iterations=100, init_basic=None, pert_eps=3.1):
    """
    Implementation of Revised Simplex Algorithm
    """
    i=1    
    m, n = A.shape
    
    # Initialize Basic and Non-Basic Sets
    II = set([j for j in range(n)])
    if init_basic is None:
        BB = set([j for j in range(n-m, n)])
    else:
        BB = init_basic
    NN = II.difference(BB)   
    x = np.zeros(n)
    prev_bases = []
    is_degenerate = False    
    
    while i < max_iterations:
        sBB = sorted(BB)
        sNN = sorted(NN)
        B = A[:, sBB]
        N = A[:, sNN]
        cB = c[sBB]
        cN = c[sNN]
        BI = np.linalg.inv(B)
        
        # Detect Dengeneracy
        if BB in prev_bases:
            print(BB)
            if not is_degenerate:
                b = b + np.matmul(B, np.power(np.ones(m)*pert_eps, np.arange(1, m+1)).transpose()) # Add Perturbation
                is_degenerate = True
        prev_bases.append(BB.copy())
        
        xB = np.matmul(BI, b)
        x = np.zeros(n)
        x[sBB] = xB
        l = np.matmul(BI.transpose(), cB)
        sN = cN - np.matmul(N.transpose(), l)
        if min(sN) >= 0:
            return x, i, np.dot(x, c), BB, True # Converged
        else:
            q = np.argmin(sN)
            Q = A[:, sNN[q]]
            d = np.matmul(BI, Q)
            if max(d) <= 0:
                return x, i, np.dot(x, c), BB, False # Unbounded
            else:
                ratios = xB/d
                pr = np.where(ratios >= 0)[0]
                p = pr[ratios[pr].argmin()]
                BB = BB.difference({sBB[p]}).union({sNN[q]})
                NN = II.difference(BB)
        i+=1
    sBB = sorted(BB)
    B = A[:, sBB]
    BI = np.linalg.inv(B)
    xB = np.matmul(BI, b)
    x = np.zeros(n)
    x[sBB] = xB
    return x, i, np.dot(x, c), BB, True # Iterations Completed
            
    

## 3(c) LP Solution

We find the initial basic feasible set and use this input for our original LP formulation.

In [4]:
# LP Problem Input

# Starting the Simplex Method (Finding the initial basic feasible set using technique from [2]) 
A = np.array([[0.86974, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, -1]])
b = np.array([100, 500, 100, -600, -500, 200, 600, -900]).transpose()
c = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]).transpose()
x, i, obj, BB, found = revised_simplex(c, A, b, 40, {25, 26, 27, 28, 29, 30, 31, 32}) # Use BB obtained as initial set

# Solving Actual LP Problem
A = np.array([[0.86974, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1]])
b = np.array([100, 500, 100, -600, -500, 200, 600, -900]).transpose()
c = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.510552, 0, -1.005]).transpose()

# Call Revised Simplex
x, i, obj, BB, found = revised_simplex(c, A, b, 100, BB)

# Print Results
print(x, i, obj, BB, found)

[1139.66686273    0.            0.          891.21385719    0.
    0.          247.21692093    0.            0.            0.
    0.            0.          451.54699446    0.            0.
  805.3517239     0.            0.          460.92547698  582.74403242
    0.            0.            0.            0.          454.02586322] 1 -456.295992539817 {0, 18, 3, 19, 6, 24, 12, 15} True


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide


## 3(d) Other Packages

In [5]:
# Call Scipy LP Solver
result = linprog(c, A_eq=A, b_eq=b, method='Simplex')

# Print Results
print(result)

     con: array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -5.68434189e-14,  1.13686838e-13,  0.00000000e+00,  0.00000000e+00])
     fun: -456.29599253981667
 message: 'Optimization terminated successfully.'
     nit: 8
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1139.66686273,    0.        ,    0.        ,  891.21385719,
          0.        ,    0.        ,  247.21692093,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
        451.54699446,    0.        ,    0.        ,  805.3517239 ,
          0.        ,    0.        ,  460.92547698,  582.74403242,
          0.        ,    0.        ,    0.        ,    0.        ,
        454.02586322])


### References

1. http://www.columbia.edu/itc/sipa/U6033/client_edit/lectures/lec4.pdf
2. Nocedal, Jorge, and Stephen Wright. Numerical optimization. Springer Science & Business Media, 2006.
